# 코사인 유사도를 위한 텍스트 전처리
재료내용 컬럼을 전처리하는 과정을 담았습니다.

In [50]:
import pandas as pd
from tqdm import tqdm

In [51]:
!pip install fastparquet

##  데이터 수 줄이기
전체 11만개의 데이터를 가지고 StandardScaler나 consine_similarity() 함수를 사용하면  
램 관련 MemoryError가 나오는 것을 확인할 수 있었습니다.  
이러한 Error를 막기 위하여 데이터 수를 줄이는 작업을 진행했습니다.

In [52]:
try:
    ori_path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\만개의 레시피.csv'
except:
    ori_path = '/content/drive/MyDrive/CP1/만개의 레시피.csv'
ori = pd.read_csv(ori_path, encoding='cp949')
ori = ori.dropna()
ori = ori.reset_index(drop=True)
ori.shape

(114512, 18)

In [53]:
try:
    craw_path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\ingredient.parquet'
except:
    craw_path = '/content/drive/MyDrive/CP1/ingredient.parquet'
craw = pd.read_parquet(craw_path, engine='fastparquet')
craw.shape

(114512, 2)

In [54]:
df = pd.concat([ori, craw['ingredient']], axis=1)
df.shape

(114512, 19)

In [55]:
df['CKG_INBUN_NM'].unique()

array(['2인분', '1인분', '3인분', '6인분이상', '4인분', '5인분'], dtype=object)

전체 데이터 중 1~2인분의 데이터만 추출하였습니다.

In [56]:
df = df.loc[(df['CKG_INBUN_NM']=='1인분') | (df['CKG_INBUN_NM'] == '2인분')]
df.shape

(59022, 19)

In [57]:
df['year'] = df['FIRST_REG_DT'].apply(lambda x : int(str(x)[:4]))
df.shape

(59022, 20)

In [58]:
df['INBUN_NUM'] = df['CKG_INBUN_NM'].apply(lambda x : int(x[0]))

In [59]:
df.shape

(59022, 21)

조회수의 25% 수준은 약 2000정도의 값이 나왔고,  
2018년도 이전은 조회수 2000이 넘지 않은 레시피는 인기가 없다고 판단하여 제거하였습니다.  
2018년도 부터는 모든 데이터를 사용하였습니다.

In [60]:
copy1 = df.loc[(df['year'] <= 2017) & (df['INQ_CNT'] > 2000)].copy()
copy2 = df.loc[(df['year'] > 2017)]
new_df = pd.concat([copy1, copy2])

In [61]:
new_df.shape

(57135, 21)

## 원본데이터에서 재료만 추출
사용자가 정의한 레시피의 재료만 사용하기 위하여 재료만 추출하는 작업을 진행하였습니다.

In [62]:
try:
    ori_path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\만개의 레시피.csv'
except:
    ori_path = '/content/drive/MyDrive/CP1/만개의 레시피.csv'
ori_df = pd.read_csv(ori_path, encoding='cp949')
ori_df = ori_df.dropna()
ori_df = ori_df.reset_index(drop=True)
ori_df.shape

(114512, 18)

In [63]:
ori_copy = ori_df['CKG_MTRL_CN'].copy()
ori_copy = ori_copy.dropna()
ori_copy

(114512,)

#### 소괄호를 제거하는 함수
데이터를 확인해보면 소괄호가 붙어있습니다.  
해당 소괄호를 제거하여 양파(소) -> 양파 로 통일할 수 있도록 합니다.

In [64]:
from collections import deque
def remove_small_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '(':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ')':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

#### 대괄호를 제거하는 함수
전체 데이터를 돌면서 대괄호를 파악하여 '[양념]'을 포함하여 뒷 부분을 제거합니다.  
데이터의 모든 '[재료]'를 포함한 모든 대괄호를 제거합니다.

In [65]:
from collections import deque
def remove_big_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '[':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ']':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

In [66]:
for i in tqdm(range(len(ori_copy))):
    temp = []
    for j in range(len(ori_copy[i])):
        if ori_copy[i][j] =='[':
            temp.append(j)
    end_index = len(ori_copy[i])
    if len(temp) > 1:
        end_index = temp[1]
    ingre = ori_copy[i][:end_index]
    ingre = remove_big_brackets(ingre).strip()
    ori_copy[i] = ingre

100%|██████████| 114512/114512 [01:00<00:00, 1896.69it/s]


새로운 컬럼을 생성해줍니다.

In [67]:
ori_df['only_ingred'] = None
ori_df['only_ingred']

0         None
1         None
2         None
3         None
4         None
          ... 
114507    None
114508    None
114509    None
114510    None
114511    None
Name: only_ingred, Length: 114512, dtype: object

#### 소괄호 한쪽을 제거하는 함수

'|'기준으로 문장을 나누고, 나누어진 리스트에서 문자를 추출하여 양 옆 공백을 제거해준뒤  
문자에 포함되어 있는 소괄호를 제거해줍니다.

다시 공백을 기준으로 문자를 나눈 후에  
생성된 리스트의 길이가 3개가 이상이라면 마지막(단위부분)을 제외하고 추출합니다.  
만약 3개 이하라면 앞에 있는 데이터만 추출합니다.

In [68]:
#한쪽만 존재하는 소괄호를 제거하는 함수
def remove_one_bracket(word):
    if word.find('(') > -1:
        idx = word.find('(')
        return word[:idx]
    if word.find(')') > -1:
        idx = word.find(')')
        return word[:idx]
    return word

In [69]:
word = '소금(asd'
remove_one_bracket(word)

'소금'

In [70]:
for i in tqdm(range(len(ori_copy))):
    lst =[]
    for word in ori_copy[i].split('|'):
        word = word.strip()
        word = remove_small_brackets(word)
        word = remove_one_bracket(word)
        li = word.split()
        if li != []:
            if len(li) < 3:
                lst.append(li[0])
            else:
                string = ' '
                lst.append(string.join(li[:-1]))
    ori_df['only_ingred'][i] = lst

  0%|          | 0/114512 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\3544315013.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ori_df['only_ingred'][i] = lst
100%|██████████| 114512/114512 [00:32<00:00, 3556.06it/s]


In [71]:
ori_df[['CKG_MTRL_CN','only_ingred']].head()

,CKG_MTRL_CN,only_ingred
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,"[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]"
2,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루]"
3,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,[파스타면]
4,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


어떤 재료들이 csv파일에 존재하는지 확인합니다.

In [72]:
ingre_list = []
for i in range(len(ori_df['only_ingred'])):
    for word in ori_df['only_ingred'][i]:
        ingre_list.append(word)

어떤 재료가 가장 많이 등장했는지 확인합니다.

In [73]:
from collections import Counter
counter = Counter(ingre_list)
counter.most_common(10)

[('양파', 31241),
 ('대파', 19334),
 ('소금', 18672),
 ('물', 13413),
 ('당근', 13313),
 ('설탕', 12938),
 ('계란', 11724),
 ('청양고추', 10521),
 ('참기름', 8904),
 ('달걀', 8749)]

## 크롤링으로 모은 재료 데이터 처리
원본 데이터를 전처리하기에는 너무 다양한 예외상황들이 존재하여  
레시피의 일련번호를 기준으로 해당 레시피 페이지에서 재료를 추출했습니다.

In [74]:
try:
    path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\ingredient.parquet'
except:
    path = '/content/drive/MyDrive/CP1/ingredient.parquet'
data_df = pd.read_parquet(path, engine='fastparquet')
data_df.shape

(114512, 2)

위에서 작업한 원본 데이터에서 재료만 추출한 데이터와 합치겠습니다.

In [75]:
data_df = pd.concat([data_df, ori_df['only_ingred']], axis=1)
data_df.shape

(114512, 3)

In [76]:
#join 하기 위하여 컬러명 변경
new_copy = new_df.rename(columns={'RCP_SNO' : 'id'}).copy()
new_copy=new_copy['id']
new_copy = pd.DataFrame(new_copy)
new_copy.shape

(57135, 1)

In [77]:
data_df = pd.merge(left=new_copy, right=data_df, how='inner', on='id')
data_df.shape

(57135, 3)

In [78]:
data_df.isnull().sum()

id             0
ingredient     0
only_ingred    0
dtype: int64

데이터를 수집할 때 오류가 나는 데이터를 빈 리스트([ ])로 수집했습니다.

ingredient컬럼에 빈 리스트([ ])가 몇개 있는지 확인하겠습니다.

In [79]:
empty_index = []
for i in range(len(data_df)):
  if data_df['ingredient'][i] == []:
    empty_index.append(i)
len(empty_index)

322

322 빈 리스트([ ])를 확인할 수 있습니다.  
모두 제거하겠습니다.

In [80]:
data_df = data_df.drop(empty_index).reset_index(drop=True)
data_df.shape

(56813, 3)

In [81]:
data_df['ingredient']

0        [[어묵, 2개], [김밥용김, 3장], [당면, 1움큼], [양파, 1/2개], ...
1        [[현미, 4컵], [찹쌀, 2컵], [호두, 50g], [물, 1/2컵], [소금...
2        [[북어포, 1마리], [찹쌀가루, 1C], [간장, 2T], [설탕, 1T], [...
3        [[파스타면], [토마토, 1개], [토마토 페이스트, 3T], [양파, 1/2개]...
4        [[건표고버섯, 9개], [오이, 1/2개], [당근, 1/2개], [양파, 1/2...
                               ...                        
56808    [[감자, 1개], [대파, 1/5개], [식용유, 2T], [소금, 1.3T], ...
56809    [[양파, 1/3개], [배추, 2잎], [당근, 조금], [청경채, 1포기], [...
56810    [[순대, 400g], [청양고추, 1개], [계란, 3개], [소금, 2꼬집], ...
56811    [[양배추, 1/4통], [양파, 1/2개], [편마늘, 1숟가락], [식용유, 적...
56812    [[식빵, 4장], [감자, 2개], [양파, 1/4개], [피망, 1/4개], [...
Name: ingredient, Length: 56813, dtype: object

In [82]:
data_df.sample(3)

,id,ingredient,only_ingred
11585,6756277,"[[계란, 4개], [강력분, 140그램], [설탕, 130그램], [꿀, 10그램...","[계란, 강력분, 설탕, 꿀, 우유, 오일, 소금, 바닐라익스트릭, 럼주]"
21459,6852527,"[[팽이버섯, 1봉지], [대파, 3대], [오이, 2개], [닭가슴살, 200g]...","[팽이버섯, 대파, 오이, 닭가슴살, 고추기름]"
31697,6885081,"[[베이컨, 5장], [영양부추, 적당량]]","[베이컨, 영양부추]"


In [83]:
for i in range(len(data_df[:10])):
  print(data_df['ingredient'][i])

[['어묵', '2개'], ['김밥용김', '3장'], ['당면', '1움큼'], ['양파', '1/2개'], ['당근', '1/2개'], ['깻잎', '6장'], ['튀김가루', '1컵'], ['올리브유', '적당량'], ['간장', '1T'], ['참기름', '1T']]
[['현미', '4컵'], ['찹쌀', '2컵'], ['호두', '50g'], ['물', '1/2컵'], ['소금', '약간']]
[['북어포', '1마리'], ['찹쌀가루', '1C'], ['간장', '2T'], ['설탕', '1T'], ['물', '1T'], ['다진파', '1T'], ['다진마늘', '1T'], ['참기름', '1T'], ['깨소금', '1T'], ['후춧가루', '약간']]
[['파스타면'], ['토마토', '1개'], ['토마토 페이스트', '3T'], ['양파', '1/2개'], ['다진마늘', '1T'], ['피망', '1/2개'], ['올리브유', '3T']]
[['건표고버섯', '9개'], ['오이', '1/2개'], ['당근', '1/2개'], ['양파', '1/2개'], ['사과', '1/2쪽'], ['그외의                                                                                    야채'], ['과일'], ['녹말가루', '2C'], ['물', '1C'], ['계란 노른자', '1개'], ['물', '2C'], ['설탕', '1/2C'], ['식초', '3T'], ['간장', '1T'], ['녹말물', '2T']]
[['닭', '1마리'], ['통마늘', '3개'], ['다진마늘', '1T'], ['화이트와인또는우유'], ['올리브유또는버터'], ['마늘솔트또는소금'], ['파인애플', '2쪽'], ['마요네즈', '2T'], ['머스터드', '1T'], ['식초', '1/2T(또는 레몬즙)'], ['꿀', '1T']]
[['양파', '1개'], ['빵가루'], ['달걀 노른자',

## 공백이 84(엄청~~긴) 데이터, 단위가 없는 데이터, [](빈리스트) 처리

```python

[['쌈배추 작은것                                                                                    4~5장'], ['달래'], ['김'], []]
```

- 공백이 긴 데이터는 공백을 기준으로 나누고
- 단위가 없는 데이터는 1을 추가해줍니다.
- 빈리스트([]) 데이터는 제거해줍니다.


In [84]:
lists = [['쌈배추 작은것                                                                                    4~5장'], ['달래'], ['김'], []]

In [85]:
def space_nounits(data):
  space = ' '*84 #84는 공백의 길이
  lists = data.copy()
  if [] in lists:
    lists.remove([])
  for i in range(len(lists)):
    if len(lists[i]) == 1 and space in lists[i][0] :
      lists[i] = lists[i][0].split(space)
    elif len(lists[i]) == 1 and lists[i] != []:
      lists[i].append(str('1단위'))
  return lists

In [86]:
print(lists)
print('-'*100)
space_nounits(lists)

[['쌈배추 작은것                                                                                    4~5장'], ['달래'], ['김'], []]
----------------------------------------------------------------------------------------------------


[['쌈배추 작은것', '4~5장'], ['달래', '1단위'], ['김', '1단위']]

In [87]:
lists2 = data_df['ingredient'][68].copy()
lists2

[['꼬막', '1봉지'],
 ['양파', '1/2개'],
 ['청.홍고추 각                                                                                    1개'],
 ['대파', '1대'],
 ['당근', '아주조금'],
 ['애호박', '1/4개'],
 ['고추장', '1큰술'],
 ['고추가루', '1큰술'],
 ['다진마늘', '1작은술'],
 ['진간장', '1작은술'],
 ['청주(맛술)', '1큰술'],
 ['요리당', '1/2작은술'],
 ['후추가루', '조금'],
 ['참기름', '1작은술'],
 ['통깨', '1큰술']]

In [88]:
space_nounits(lists2)

[['꼬막', '1봉지'],
 ['양파', '1/2개'],
 ['청.홍고추 각', '1개'],
 ['대파', '1대'],
 ['당근', '아주조금'],
 ['애호박', '1/4개'],
 ['고추장', '1큰술'],
 ['고추가루', '1큰술'],
 ['다진마늘', '1작은술'],
 ['진간장', '1작은술'],
 ['청주(맛술)', '1큰술'],
 ['요리당', '1/2작은술'],
 ['후추가루', '조금'],
 ['참기름', '1작은술'],
 ['통깨', '1큰술']]

In [89]:
copy_data =data_df.copy()
copy_data.head()

,id,ingredient,only_ingred
0,128671,"[[어묵, 2개], [김밥용김, 3장], [당면, 1움큼], [양파, 1/2개], ...","[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,131871,"[[현미, 4컵], [찹쌀, 2컵], [호두, 50g], [물, 1/2컵], [소금...","[현미, 찹쌀, 호두, 물, 소금]"
2,139247,"[[북어포, 1마리], [찹쌀가루, 1C], [간장, 2T], [설탕, 1T], [...","[북어포, 찹쌀가루]"
3,149207,"[[파스타면], [토마토, 1개], [토마토 페이스트, 3T], [양파, 1/2개]...",[파스타면]
4,151148,"[[건표고버섯, 9개], [오이, 1/2개], [당근, 1/2개], [양파, 1/2...","[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


In [90]:
#test 6분 정도 소요
from tqdm import tqdm
for i in tqdm(range(len(copy_data['ingredient']))):
  try : 
    copy_data['ingredient'][i] = space_nounits(copy_data['ingredient'][i])
  except:
    print(i)

  0%|          | 0/56813 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\3921183491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = space_nounits(copy_data['ingredient'][i])
100%|██████████| 56813/56813 [01:03<00:00, 898.75it/s] 


In [91]:
#랜덤 인덱스 추출하여 확인
random_index = copy_data.sample(10).index
for index in random_index:
  print(copy_data['ingredient'][index])

[['떡국떡 떡볶이떡', '300g'], ['사각어묵', '4장'], ['대파', '1대'], ['계란', '2알'], ['고추장', '4Ts'], ['설탕', '3Ts'], ['고춧가루', '2Ts'], ['쇠고기다시다', '1ts'], ['다진마늘', '1ts'], ['식용유', '1Ts'], ['후추', '약간'], ['물', '800ml']]
[['오이', '1개'], ['양파', '1/4개'], ['설탕', '1/2T'], ['고춧가루', '1+1/2스푼'], ['식초', '2T'], ['간장', '1T'], ['간마늘', '1t'], ['파', '1움큼'], ['통깨', '약간']]
[['황태포', '1줌'], ['대파', '1/2뿌리'], ['청양고추', '2개'], ['간장', '1.5T'], ['고추장', '2T'], ['설탕', '1T'], ['올리고당', '1/2T'], ['다진마늘', '1T'], ['참기름', '1T'], ['깨', '원투솔솔']]
[['파스타', '2인분'], ['동죽', '1컵'], ['슬라이스마늘', '1스푼'], ['올리브 오일', '1/2컵'], ['파슬리', '톡톡'], ['마늘', '3쪽'], ['후추', '톡톡']]
[['떡볶이떡', '1단위'], ['비엔나소시지', '1단위'], ['참기름or들기름', '1단위'], ['고추장', '1'], ['케첩', '2'], ['올리고당', '2'], ['간장', '1'], ['설탕', '1'], ['고추가루', '1'], ['물', '1/2종이컵']]
[['감자', '1개'], ['버터', '1큰술'], ['파슬리', '약간'], ['소금', '약간']]
[['세발나물', '180g'], ['조미김가루', '1컵 정도'], ['진간장', '1작은술'], ['미림', '1작은술'], ['참기름', '2작은술'], ['굵은 고춧가루', '1작은술'], ['통깨', '1/2큰술']]
[['백다대기 오이', '2개'], ['홍고추', '1개'], ['소금', '1꼬집'],

## 소괄호() 제거

In [92]:
from collections import deque
def remove_small_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '(':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ')':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

In [93]:
word = '약간 (적당히)'
remove_small_brackets(word)

'약간 '

문자에 들어있는 소괄호를 제거하겠습니다.

In [94]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        if copy_data['ingredient'][i][j] != []:
            copy_data['ingredient'][i][j][0] = remove_small_brackets(copy_data['ingredient'][i][j][0])
            copy_data['ingredient'][i][j][1] = remove_small_brackets(copy_data['ingredient'][i][j][1])

100%|██████████| 56813/56813 [00:24<00:00, 2300.21it/s]


### 재료부분 소괄호 확인

In [95]:
count = 0
ingredient_list = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        if data != []:
           if '(' in str(data[0]) or ')' in str(data[0]):
                ingredient_list.append(data[0])
print(len(ingredient_list))

100%|██████████| 56813/56813 [00:05<00:00, 11202.56it/s]

325


In [96]:
for i in range(len(ingredient_list)):
    word = ingredient_list[i]
    print(word)

명란젓(없어도
빵(식빵이나
칼라별 피망 약간씩(
닭(
참치액 조금 (없으면
고추장 육회 양념장(
무(
당근(
칠리소스(없어도
고구마중간 사이즈 1개)
호떡믹스 (잼믹스
(다시마
식빵(두께
다시마(채썬
식물성오일(식용유
육수치킨육수)
2차 발효 후 반죽에 바를 계란물 (노른자
맛살(또는햄
참치)
참기름(또는 고추씨
데미글라스 소스 2T(또는 굴소스
달걀물(바르는
피자치즈 약간(혹은
매실청(설탕
당근中(
초록색파프리카中(
재래간장(재래간장이 없으시면 간장을 조금 더 넣어 주세
초간장 (진간장
마요네즈(마요네즈를 사용할 경우 손에 조금 질척거림이
설탕(설탕을 넣으실 경우 물 1큰술을 더 넣어 주세요.
스리라차 칠리소스입니다. 약간 매운편입니다.)
바질 약간 (일반 소고기패티
대파 흰대(
아몬드(생략
영양부추 한줌(
파프리카 (
빼빼로 스틱 32개 (
오이피클(익은
리코타치즈 블럭(까망베르나 브리치즈 슬라이스 치즈대체가
키위(골드키위
그린키위)
뒷다리살)
소고기밑간(청주
연유(또는
설탕)
다시마(5X
육수(다시마와
피망(오이나
로메인(상추 대체
비트무(또는
항암배춧잎(또는
채소(항암 배춧잎/비트잎/붉은
호떡믹스 (잼믹스
우유(얼린것) 적당량(
호떡믹스 (잼믹스
달걀지단(달걀
매실액 (or
당근 약간(약
양파 약간(약
파 약간 (약
양파 약간(약
호떡믹스 (잼믹스
건다시마 (3cmX
호떡믹스 (잼믹스
들기름(or
각종채소(양배추/적채/당근/깻잎/양파
허브솔트 (or 스테이크
도라지 한 줌 (대략
호떡믹스 (잼믹스
육수(다시마
노란색)
닭한마리(
대파(파란 부분만
물(녹두의
분홍소세지(옛날
식용유(취향껏 조절
물 적당히(재료 잠길
식용유 (취향
청량고추(기호에 맞게
매실액 3T(매실액이 없으면 물엿
맛소금 약간(
요거트(두유
삶은 고사리(건고사리이면 불려서
올리브유)
밥(
떡볶이소스 6큰술 (고추장
딸기쨈(생략
돼지고기(명절에
대파(냉장고 속
(계란
대파(쫑쫑
청고추(생략
가쓰오부시(또는
) 검은
파프리카)
물 (냄비에
바닐라빈 약간(
콩나물 (냉장고에 있는 야채들
쌀 

위처럼 괄호가 한쪽씩 있는 데이터가 존재하는 것을 확인할 수 있습니다.

### 단위부분 소괄호 확인

In [97]:
unit_list = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        if data != []:
           if '(' in str(data[1]) or ')' in str(data[1]):
                unit_list.append(data[1])
print(len(unit_list))

100%|██████████| 56813/56813 [00:05<00:00, 11216.79it/s]

326


In [98]:
for i in range(len(unit_list)):
    word = unit_list[i]
    print(word)

무방)
1줌(기름기가 적은 앞다리살을 이용했어
모닝빵등)
40g)
480g)1/2마리
진간장)
5cm)
300g 기준)
2컵(물 3컵+멸치6마리+다시마 5X5
5마리( 칵테일새우 이용하면 더 좋아요
4센티)1/2토막
4센티)1/2토막
1개(클수록 국물이 더 진하고 맛있어요
되요)
2cm)
1캔((200g)
것)
+물)
기름)
1T)
용)
100g)
1/2큰술)1큰술
50g)1/2개
30g)1/2개
약간)
1개(길이가 12cm 정도의 통통한 고
50g(생크림이 없으시면 그냥 우유를 넣
3~4개분) - 이것중 하나만 사용할꺼랍니
5cm)
가능)
1스푼+ 물1/3컵)
50g)
34g~취향껏)
0.5~0.8mm 두께 10~15cm 길이)
배추김치)
80ml(우유 75ml + 식초 5ml
2/3Ts)
5)2.5개
1스푼(커피용스푼
건새우)
시금치)
가능)
단무지)
깻잎)
100g(20*20 정사각틀 1개분량 10
파프리카)
500ml)
2개)
설탕)
10g)
10g)
10g)
10g)
1수저(어른밥수저기준
4cm)2-3장
포도씨유)
등)
시즈닝)
150g)
1.2kg)
1뿌리)
5배이상)
5컵(1000
1/2큰술(or소주or청하등의순도낮은술1큰
1t(혹은 참기름과 고추기름 반반씩 섞
1/2컵(반죽농도에따라 추가로 더 넣을수있
소세지)
가능)
정도)
것)
것)
생략가능)
2개(껍질과 씨 제거한 무게 650g
1/4작은술(없으면 진간장 티스푼으로 약간
1ts 좀 안되게)
요거트)
삶기)
1인분)
1개(100g. 소고기50 돼지고기50
3큰술)
9컵(쌀을 불린 시간이나 잡곡의 유무에
가능)
삶은)
채소)
1개)
1냉면그릇(끓이는 냄비에 따라 조절가능
썬것)
가능)
혼다시)
파슬리(생략가능
1T)
2/3정도 찰 정도)
1/5정도)
모아모아~)
1+1/2공기 정도)
대체가능)
6그릇(밥솥살때 주는 컵으로 4컵했어요
2큰 술반)
1/4개)50g
200g)
1호)
2큰술)
500원 동전보다 살짝 크게 움켜쥔 정도(
야채)
가능)
등등)
200g)
떡갈비)
4사이즈)3-4장
1큰술)

단위부분 또한 소괄호가 한쪽씩 들어가 있는 데이터가 존재합니다.

### 한쪽만 있는 소괄호 제거

괄호가 한쪽밖에 없는 뒷부분을 제거합니다.

In [99]:
test_sample = [
    [['어묵', '2개'],['김밥용김', '3장'],['당(면', '1움큼'], ['양파', '1/2개'],['당근', '1/2개'], ['깻잎', '6장'],['튀김가루', '1)컵'], ['올리(브유', '적당량'], ['간장', '1T'],['참기)름', '1T']],
    [['현미', '4컵'], ['찹)쌀', '2컵'], ['호두', '50g'], ['물', '1/2(컵'], ['소(금', '약간']]
]
for i in range(len(test_sample)):
    lst = []
    for j in range(len(test_sample[i])):
        data = test_sample[i][j]
        for k in range(len(data)):
            if data[k].find('(') > -1:
                idx = data[k].find('(')
                data[k] = data[k][:idx]
            if data[k].find(')') > -1:
                idx = data[k].find(')')
                data[k] = data[k][:idx]
            
test_sample      

[[['어묵', '2개'],
  ['김밥용김', '3장'],
  ['당', '1움큼'],
  ['양파', '1/2개'],
  ['당근', '1/2개'],
  ['깻잎', '6장'],
  ['튀김가루', '1'],
  ['올리', '적당량'],
  ['간장', '1T'],
  ['참기', '1T']],
 [['현미', '4컵'], ['찹', '2컵'], ['호두', '50g'], ['물', '1/2'], ['소', '약간']]]

In [100]:
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        for k in range(len(data)):
            if data[k].find('(') > -1:
                idx = data[k].find('(')
                data[k] = data[k][:idx]
            if data[k].find(')') > -1:
                idx = data[k].find(')')
                data[k] = data[k][:idx]

100%|██████████| 56813/56813 [00:05<00:00, 9939.23it/s] 


In [101]:
lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        if data != []:
            if ('(' in data[0]) or(')' in data[0]) or ('(' in data[1]) or (')' in data[1]):
                lst.append(data)
print(len(lst))

100%|██████████| 56813/56813 [00:04<00:00, 11433.91it/s]

0


괄호가 없는 것을 확인할 수 있습니다.

마지막으로 데이터에 소괄호가 있는지 확인해보겠습니다.

In [102]:
#랜덤 인덱스 추출하여 확인
random_index = copy_data.sample(10).index
for index in random_index:
  print(copy_data['ingredient'][index])

[['메밀국수', '2인분'], ['김치', '1/4포기'], ['김칫국물', '1국자'], ['고추장', '1작은술'], ['설탕', '1작은술'], ['매실액', '1큰술'], ['탄산수', '1작은통'], ['들기름', '1작은술'], ['파프리카', '1단위'], ['오이', '1단위'], ['당근', '1단위'], ['통깨', '1단위']]
[['새우', '10-15마리'], ['옥수수전분', '100g'], ['소금', '1/4스푼'], ['오레가노', '1스푼'], ['양배추', '1/4통'], ['식용유', '1리터'], ['마늘', '3알'], ['고추', '1/2개'], ['설탕', '1-2스푼'], ['라임', '1개'], ['피쉬소스', '4-5스푼'], ['민트', '5-10g']]
[['사이다', '1컵'], ['100%오렌지주스', '1컵'], ['얼음', '약간'], ['레몬즙', '1큰술']]
[['만두', '8개'], ['흑미떡국떡', '1단위'], ['사골곰탕', '500ml'], ['달걀', '1개'], ['파', '적당량'], ['후추', '약간'], ['참기름', '약간']]
[['식빵', '1개'], ['버터', '3조각'], ['팥앙금', '3~4큰술'], ['볶은 콩가루', '1/3T'], ['구운 슬라이스 아몬드', '조금']]
[['물미역 한팩', '201g'], ['국거리소고기 한줌', '75g'], ['물', '800ml'], ['참기름', '1큰술'], ['다진마늘', '1/2큰술'], ['국간장', '2큰술'], ['대파', '1단위']]
[['김치', '1공기'], ['파', '50g'], ['스팸', '3조각'], ['밥', '2공기'], ['간장', '2스푼'], ['굴소스', '1스푼'], ['고춧가루', '2스푼']]
[['불고기', '400g'], ['양파', '1단위'], ['당근', '1단위'], ['대파', '1단위'], ['양송이버섯', '1단위'], ['간장', '10큰술'], ['올리

## 양념부분제거(재료만 추출해서)

크롤링으로 모은 재료 데이터에서 양념부분을 제외한 실제 재료 데이터만 추출하겠습니다.

In [103]:
all_lst = []
for i in tqdm(range(len(copy_data))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if len(data) == 0:
            pass
        else:
            if len(copy_data['only_ingred'][i]) == 0:
                    lst.append(data)
            for word in copy_data['only_ingred'][i]:
                if data[0] in word:
                    lst.append(data)
    all_lst.append(lst)

100%|██████████| 56813/56813 [00:09<00:00, 5986.36it/s]


In [104]:
copy_data['ingredient'] = pd.Series(all_lst)

In [106]:
n = 1000
print(copy_data.iloc[n,]['ingredient'])
print(copy_data.iloc[n,]['only_ingred'])

[['네모오뎅', '1단위'], ['양파', '1단위'], ['매운고추', '1단위']]
['네모오뎅', '양파', '매운고추']


## 문자안에 특수문자 확인
- '-'
- '~'
- '/'
- '+'
-....

위의 특수문자를 제외하고 다양한 특수문자가 들어가 있습니다.  
저희가 다루어야할 주요 특수문자는 위의 4가지로 정의하였고,  
해당 특수문자를 중심으로 전처리를 진행하겠습니다.

전처리는 재료부분과 단위부분을 따로 다루게 됩니다.

### '-' 이 들어가 있는 데이터 확인

In [107]:
ingre_dash_set = set()
ingre_dash_index_set = set()
unit_dash_set = set()
unit_dash_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '-' in data[0]:
                ingre_dash_set.add(data[0])
                ingre_dash_index_set.add(i)
            if '-' in data[1]:
                unit_dash_set.add(data[1])
                unit_dash_index_set.add(i)
print(len(ingre_dash_set))
print(len(ingre_dash_index_set))
print(len(unit_dash_set))
print(len(unit_dash_index_set))  

100%|██████████| 56813/56813 [00:00<00:00, 88624.73it/s]

32
26
297
875


#### 재료부분

In [108]:
for word in list(ingre_dash_set):
    print(word)

선택 - 양파
육우 - 홍두깨살
다슬기 무침용 -아욱
녹말가루-혹은 감자전분
물 약간 등 양념장- 고추장
다슬기 무침용 -밀가루
생잎새버섯 2줌 양념장- 쑥갓 다진 것
라따뚜이 - 토마토
얼음 탁구공만한것-! 큰덩어리얼음없음 각얼음 큰
물 150ml-
쿠키바닥-버터
아이싱 - 슈거파우더
김채 1줌 등 양념장- 진간장
육수 - 물
알리오올리오 - 스파게티면
삶을 때 - 물
쫑쫑 썬 대파- 듬뿍
깍둑썰기한 양파- 듬뿍
쿠키바닥-다이제
도우 - 우리 밀가루강력분
미니 바게트 - 우리밀 중력분
K-MILK 우유
딸기바나나쉐이크-바나나
육수 - 멸치
꽈리고추 한봉- 약
--버섯
다슬기 무침용 -달걀
돼지고기 안심-
Semi-Dried 방울토마토
칵테일새우-중간이상사이즈
베이컨 or 스팸 베이컨-
양파-채썬것


#### 재료부분처리

'-' 기준으로 나누고 앞에 있는 데이터를 사용하겠습니다.

In [109]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '-' in data[0]:
                data[0] = data[0].split('-')[0].strip()

100%|██████████| 56813/56813 [00:00<00:00, 94843.47it/s]


In [110]:
space_idx = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] == '':
                space_idx.append(i)

100%|██████████| 56813/56813 [00:00<00:00, 95090.08it/s]


In [111]:
for i in tqdm(space_idx):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] != '':
                lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/181 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\1768163441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 181/181 [00:00<00:00, 4022.28it/s]


In [112]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] == '':
                print(data, i)

100%|██████████| 56813/56813 [00:00<00:00, 93914.41it/s]


#### 단위부분

In [113]:
for word in list(unit_dash_set):
    print(word)

4-5줄기
10-15마리
80-90ml
10-15g
3-4방울
3-4송이
6-7T
2장 스팸-1/4
4-5바퀴
4-5줄
0.5t-1t
2-3톨
5-6개
4-5TS
3-4줌
8-9스푼
4-7개
60-70알
3-4컵
1-2줄
1.5-2공기
4-5쪽
5-6숟가락
0.5-0.7T
1.5-2T
5-8줄
12-15알
5-7알
4-5큰 술
100-120ml
4-5마리
4-5알
13-15개
0.5-1T
1-2T
5-8알
1/4-1/3분량
6-7장
2-3T
50-100ml
2-4스푼
8-12쪽
2-3토막
-
10-15장
1-2줌
8-9마리
15-20ea
1-2L
15-20cm5개
5-10알
2-3이파리
10-12개 ​
4-5T
8-10개
3개-다지기
3-4쪽
600-700ml
5-7장
2-3숟가락
1-2컵
5-10개
2-3인분
2-3꼬집
2-3Tbs+소금 약간
1-2숟갈
3-4조각
1-2마리
50-60g
15-20개
20-25개
3-4장
50-60ml
4-6알
16-17마리
100-150g
200-300ml
250-300g
0.5-1국자
1-2큰 술
5-6인분용
1-2인분
1-2쪽
300-400g
500-600g
10-20ml
1-1.5T
1-2장
5-6마리
18-20개
6-7개
6-8방울
1-1.5컵
60-70g
6-7알
4-5컵
3-4편 썬것
30-40ml
3-4꼬집
1-2장씩
1-2TS
100-150g정도
50-100g정도
20-30ml
1-2큰술 등
6-8장
2-3방울
3-4스푼
1/2-1큰 술
7-8쪽
15-20알
4-6개
2-3조각
2-3줄
2-3개 취향껏
500-70ml
9-10조각
6-10개
9-10장
2-3숟갈
2-300g
8-9개
3-4큰술
2-3쪽
2-3젓가락
200-300g
3-4슬라이스
0.5-1대
1-3
20g-한움큼
4-5개
200ml-설탕으로 대체 가능
7-8알
1-2머그컵
6-10알
150-200ml
2-3TS
3-5조각
2-3장 생략가능
7-8줄
20-30알
4-5숟가락
10-15개
70-80g
5-6컵
160-200g
170-180ml
5-6

단위쪽과 재료쪽의 구조가 조금 다른것 같습니다.

단위 부분은 '-' 뒤쪽만 살리도록 하겠습니다.

#### 단위부분처리

'-' 기준으로 나누고 뒤에 있는 데이터를 사용하겠습니다.

In [114]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '-' in data[1]:
                data[1] = data[1].split('-')[1].strip()

100%|██████████| 56813/56813 [00:00<00:00, 94215.81it/s]


In [115]:
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[1] != '':
                lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/56813 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\1261346209.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 56813/56813 [00:13<00:00, 4183.76it/s]


In [116]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[1] == '':
                print(data, i)

100%|██████████| 56813/56813 [00:00<00:00, 93288.13it/s]


### '~' 들어가 있는 데이터 확인

In [117]:
ingre_wave_set = set()
ingre_wave_index_set = set()
unit_wave_set = set()
unit_wave_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '~' in data[0]:
                ingre_wave_set.add(data[0])
                ingre_wave_index_set.add(i)
            if '~' in data[1]:
                unit_wave_set.add(data[1])
                unit_wave_index_set.add(i)
                
print(len(ingre_wave_set))
print(len(ingre_wave_index_set))
print(len(unit_wave_set))
print(len(unit_wave_index_set)) 

100%|██████████| 56813/56813 [00:00<00:00, 88632.32it/s]

34
34
899
3625


#### 재료부분

In [118]:
for word in list(ingre_wave_set):
    print(word)

청양고추 3개~
대파 1개~
요구르트 약 100ml~
우유 200ml~
카놀라유 400g~
통마늘 10개~
익은김치~
소금 ~
멸치육수 400ml~
전분 2T~
냉이 1줌~
청양고추 1개~
우유 약 300ml ~
물 170ml~
0세~2세까지 아기 멸치
김치 반포기~한
각종 전~
치킨스톡 4컵~
4~5cm길이로 자른 쪽파
생수 ~
바나나 큰거 1개~중간
4~5cm길이로 자른 부추
밥 듬뿍~
채끝살 100g~
5cm~6cm정도의 두께로 자른 식빵
멸치 8개~
딸기 중~대형
양상추등~~
얼린 딸기 8개~
굴소스 2작은술~
10~15cm가량 다시마
유자청 두 큰술 ~ 세
콩나물 반봉~
오이 2개~


#### 재료부분처리

'~'문자만 제거 해 주고, ' '(space)기준으로 나누어서 앞에 있는 데이터를 사용하겠습니다.

In [119]:
for i in list(ingre_wave_index_set):
    for data in copy_data['ingredient'][i]:
        if '~' in data[0]:
            data[0] = data[0].replace('~','')
            data[0] = data[0].split()[0]

In [120]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if '~' in data[0]:
            print(data)

100%|██████████| 56813/56813 [00:00<00:00, 96293.82it/s]


In [121]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data[0] == '':
            print(data)

100%|██████████| 56813/56813 [00:00<00:00, 97611.00it/s]


#### 단위부분

In [122]:
for word in list(unit_wave_set):
    print(word)

8~10개
0.5숟갈~1숟갈
1/2~2/3공기
200~30ml
13~15개
2~3큰술
2~300g
8~10개정도 취향것
14~16개
1~1+1/2공기
2~3겹
1인분당2~3개
7~8스푼
500~600g
4~5컵
2개 ~ 취향껏
8~10
3~5톨
1/2~1T
180~200g
7~10마리
100~200g
1~1.5수저
30~35개
1~2주먹
1~2t
1~2쪽
6~8개
1/2~1큰술
8~9알
8~개
170~200ml
2~3토막
100~110ml
200~250ml
2~2+1/2개
5~6g
2~3톨
5~8마리
95~100ml
1~1.5숟가락
3~5개
1~2국자
15~16장
1~1+1/2큰숟갈
40~50장
900~1000ml
2~3대
1/4~1/5정도
15~20장
150~200ML
1~2묶음
30~50ml정도
5~10개
70~80cc
8~9cm길이
1/3~1/2t
3~3+1/2스푼
4~6마리
1/2T ~1T
5~8조각
쓸만큼~
듬~뿍
1/3~1/2T
2~3줄
1/3~1/4T
2~3Ts
2~3팩
5~6개정도
4~5큰술
3~4송이
900~100ml
3~4스푼
1/3~1/2컵
2~3줌
3~5
1~1.5큰술
10~15줄
5~7장
7~14장
1작은술~취향껏
3~4큰술이상
100~250ml
250~300
3~4뿌리
2~3잎
0.5~1T
1~1.5개
1국공기~
150~200g
1~2줌
1/2~1/3티스푼
4~500g
8~9개
5~6개 분량
2~3방울
1스픈 없으면 패스~
8~9마리
90~100g 
5~6쪽
20~25개정도
60~80g
4~5줄
2~3큰술 정도
1~2바퀴
솔솔솔~
2/3~1숟가락
15~20g
2컵~세컵
30~40개 정도
2~3쪽
40~50그램
1~1.5인분
5~6큐브
5~6줄기 정도
10~15장
2~3번톡톡
35~40개
200~300g정도
1개400~5
4~6알
2~2.5줌
1/2~2/3고구마양
10~13쪽
3~5cm
4~5알
2~4장
180ml~
3~5큰술
2~4
500~1000ml
3~4g
200~300그램
2줌~
500~700g
15~17마리
2~3큰슬
25~30

#### 단위부분처리

'~'기준으로 나누고 뒤에 있는 데이터를 사용합니다.

In [123]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if '~' in data[1]:
            data[1] = data[1].split('~')[1].strip()

100%|██████████| 56813/56813 [00:00<00:00, 96292.03it/s]


In [124]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if '~' in data[1]:
            print(data)

100%|██████████| 56813/56813 [00:00<00:00, 97117.86it/s]


In [125]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data[1] == '':
            print(data)

 70%|███████   | 40019/56813 [00:00<00:00, 99248.33it/s]

['파슬리가루', '']
['파마산 치즈가루', '']
['치즈', '']
['버터', '']
['베이컨', '']
['얼음', '']
['튀김용 식용유', '']
['소금', '']
['간 돼지고기', '']
['물', '']
['햄', '']
['아위버섯', '']
['생크림', '']
['생크림', '']
['피자 치즈', '']
['토핑용 크렌베리', '']
['굴소스', '']
['삶은 흰강낭콩', '']
['물', '']
['등갈비', '']
['생강', '']
['마요네즈', '']
['무', '']
['토마토소스', '']
['화이트소스', '']
['치킨', '']
['새우', '']
['올리고당', '']
['슬라이스치즈', '']
['슬라이스 햄', '']
['돼지등심', '']
['통모짜렐라치즈', '']
['소고기 샤브샤브용', '']
['통 모짜렐라치즈', '']
['빵가루', '']
['잡채', '']
['유자청', '']
['레몬즙', '']
['설탕', '']

100%|██████████| 56813/56813 [00:00<00:00, 99498.15it/s]


['대패삼겹살', '']


In [126]:
#all_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[1] != '':
                lst.append(data)
    #all_lst.append(lst)
    copy_data['ingredient'][i] = lst

  0%|          | 0/56813 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\2533240754.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 56813/56813 [00:13<00:00, 4180.68it/s]


In [127]:
#copy_data['ingredient'] = pd.Series(all_lst)

In [128]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data[1] == '':
            print(data)

100%|██████████| 56813/56813 [00:00<00:00, 98977.95it/s]


### '+' 들어가 있는 데이터 확인

In [129]:
ingre_plus_set = set()
ingre_plus_index_set = set()
unit_plus_set = set()
unit_plus_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                ingre_plus_set.add(data[0])
                ingre_plus_index_set.add(i)
            if '+' in data[1]:
                unit_plus_set.add(data[1])
                unit_plus_index_set.add(i)
print(len(ingre_plus_set))
print(len(ingre_plus_index_set))
print(len(unit_plus_set))
print(len(unit_plus_index_set))

100%|██████████| 56813/56813 [00:00<00:00, 89188.47it/s]

124
132
188
1550


#### 재료부분

In [130]:
for word in list(ingre_plus_set):
    print(word)

간장샐러드소스+바질
찹쌀 현미+백미
물+얼음
블루베리+복분자
물 500ml + 진간장
참기름+식용유
전분가루 2큰술+ 물
계란+소금+파
파슬리가루+훈제파프리카가루
스팸 +비엔나
청주+생강즙
피자치즈+치즈가루
양파+호박
마요+케첩
삶은 백태+ 물
설탕+
묵은지+파김치
소금 + 오일
닭 날개 + 봉
식용유+들기름 섞은것
들기름+식용유
가지+소금
참치+마요네즈
쌀뜨물+다시마
당근+소금
다시마+황태+파뿌리
김치 + 김치국물
설탕 1T+
크림+우유 섞인것
양파+식촛물
소금+오일+후춧가루+청주
소금+멸치액젓
달래+냉이
참기름+현미유
황태육수황태+멸치+다시마
황태+건표고
체다치즈 +모짜렐라치즈 약
청주 +맛술
다시마물+작두콩차
쑥+쌀가루
버터+오일
쌀뜨물+천일염
우유 150ml +
다시마 + 멸치 육수
김치국물+신김치 작은
식용유+참기름
들기름+올리브오일
굵은소금+소주
양파+식초
계란+소금
올리브유 1 Tbsp +
모짜렐라+체다치즈
간장 + 고추냉이 소스
녹인버터+오일+소금+후춧가루+파마산 치즈가루
튀김가루+빵가루
국간장+멸치액젓 기호에
부침가루+튀김가루
고추장+된장
올리브오일 1 Tbsp +
김치+김치국물
전분+찹쌀가루
케첩+핫소스
빨강+ 노랑 파프리카
우스타소스+씨겨자
백앙금+초코시럽
올리브오일+갈릭솔트+그라인드후추소고기
멸치+새우가루
삼색 하트파스타+삼색 동물모양 파스타
매실액+양파청+산야초효소
견과류 + 깨
피클+할라피뇨
황태머리+천연조미료육수
멸치+다시마
쭈꾸미+낙지
검은깨+참깨
내장+알
감자전분 +옥수수전분
햄+깻잎+브로컬리+통깨
들기름+참기름
달걀+소금
감자+고구마 약
다진 돼지고기 +다진소고기
참기름+들기름
김치+총각 김치
초간장 1T+
밀가루+계란+빵가루
비빔오징어젓갈+들기름
밀가루 1컵 +
화이트와인+레몬즙
다시마+멸치
모짜렐라 치즈+체다치즈
소금+화이트페퍼
멸치+다시마 육수
우유+생크림
튀김가루+물
체다+모짜렐라치즈
쌀+잡곡
날치알+소주
밀가루+빵가루
커피가루 1g+물
마요네즈+머스터드
검은깨+참깨 섞어서
대게다시팩+ 파뿌리
우유+맛술
밥+참기름
고추장

재료부분과 단위부분을 확인해보니  
문자의 맨앞과 맨뒤에 '+'가 붙어있는 것을 확인할 수 있었습니다.
제거해주도록 하겠습니다.

In [131]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                data[0] = data[0].strip('+')
            if '+' in data[1]:
                data[1] = data[1].strip('+')

100%|██████████| 56813/56813 [00:00<00:00, 88142.87it/s]


In [132]:
tmp = []
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                tmp.append(data[0])
print(len(tmp))

132


In [133]:
for word in tmp[:10]:
    print(word)

튀김가루+빵가루
밀가루+빵가루
삼색 하트파스타+삼색 동물모양 파스타
빨강+ 노랑 파프리카
백앙금+초코시럽
우유+커피 섞은것
검은깨+참깨
올리브유 + 녹인버터
쭈꾸미+낙지
우유+생크림


#### 재료부분처리

'+'가 들어가있는 재료는 제거하도록 하겠습니다.

In [134]:
all_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                pass
            else:
                lst.append(data)
    all_lst.append(lst)

100%|██████████| 56813/56813 [00:00<00:00, 80496.53it/s]


In [135]:
copy_data['ingredient'] = pd.Series(all_lst)

In [136]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                print(data)

100%|██████████| 56813/56813 [00:00<00:00, 95320.40it/s]


#### 단위부분

'+'가 들어간 단위부분은 밑에서 처리할 '/'가 들어간 단위부분과 많이 겹쳐서 같이 처리하겠습니다.

In [137]:
plus_unit_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[1]:
                plus_unit_lst.append(data[1])

100%|██████████| 56813/56813 [00:00<00:00, 95645.07it/s]


In [138]:
plus_unit_lst

['1+1/2개',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2줌',
 '1+1/2공기',
 '1+1/4컵',
 '1+1/2컵',
 '1+1/2공기',
 '1+1/2공기',
 '1+1/2컵',
 '1+1/2공기',
 '1+1/2공기',
 '1+1/2컵',
 '2+1/2밥공기',
 '2+1/2종이컵',
 '1+1/2컵',
 '1+1/2공기',
 '2+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2개',
 '1+1/2공기',
 '1+1/2숟갈',
 '1+1/2큰술',
 '2+1/2컵',
 '1+1/2공기',
 '2+1/2컵',
 '1+1/2컵',
 '4+1/2컵',
 '2+1/2컵',
 '1+1/2개',
 '1+1/2개',
 '1+1/2줌',
 '1+1/2줌',
 '1+1/2공기',
 '3+1/2컵',
 '3+1/2컵',
 '1+1/2종이컵',
 '1+1/2개',
 '1+1/2큰술',
 '3+1/2T',
 '2+1/2컵',
 '1+1/2큰술',
 '1+1/2컵',
 '1+1/2공기',
 '2+1/2큰술',
 '1+1/2개',
 '1+1/2개',
 '1+1/2컵',
 '1+1/2컵',
 '3+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2공기',
 '1+1/2공기',
 '1+1/2큰술',
 '1+1/2C',
 '1+1/2C',
 '1+1/2큰술',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2cc',
 '4+1/2컵',
 '1+1/2개',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2개',
 '2+1/2컵 ',
 '1+1/2공기',
 '1+1/2컵',
 '1+1/2스푼',
 '1+1/2C',
 '1+1/2컵',
 '1+1/2큰술',
 '6+1/2컵',
 '1+1/2Ts',
 '1+1/2Ts',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2뭉치',
 '1+1/2큰술',
 '1+1/2C',
 '1+1/2개',
 '1+1/2컵'

###  '/' 들어가 있는 데이터 확인

In [139]:
ingre_slash_set = set()
ingre_slash_index_set = set()
unit_slash_set = set()
unit_slash_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '/' in data[0]:
                ingre_slash_set.add(data[0])
                ingre_slash_index_set.add(i)
            if '/' in data[1]:
                unit_slash_set.add(data[1])
                unit_slash_index_set.add(i)
print(len(ingre_slash_set))
print(len(ingre_slash_index_set))
print(len(unit_slash_set))
print(len(unit_slash_index_set))

100%|██████████| 56813/56813 [00:00<00:00, 85561.74it/s]

185
207
983
25481


#### 재료부분

In [140]:
for word in list(ingre_slash_set):
    print(word)

청주/맛술
소금 /후추
1/4통 양배추
자반/조미김
사이다/콜라 조그마한
모시조개/바지락
매실액/설탕
쫄면/소면
돼지고기 목살/앞다리살
생크림 /우유/버터
흑설탕/황설탕
쪽파 /대파
물 /우유
게맛살 /크래미
고량주/독주
바질페스토/딸기잼
바나나 /
로메인/시금치/상추 등 채소
크림치즈/훼타치즈
건자두콤포트/잼
딸기시럽/딸기잼
당근/깻잎/양배추
설탕 50g/
오이지피클/매실장아찌
멸치다시마육수/천연조미료육수
깨/후춧가루
생강술/청주/미림/소주
곶감/반건시
소금/새우젓
물/다시육수
맛간장/쯔유/국탕용분말스프/혼다시
쪽파/잔파
떡국떡 /떡볶이떡
느타리/표고버섯
조청 /꿀
햄/베이컨
토마토소스/브라바소스
맛술/청주
1/8크기의조미김
참기름/들기름
알배추/시금치
청고추/청양고추
노랑/주황 파프리카
멸치육수/다시마육수/천연조미료육수/황태육수
멥쌀/현미불린것
육수/물
깻잎/대파
쇼트닝/버터
설탕 / 연유
다진 대파/쪽파
유자청/오렌지마말레이드
오징어/낙지
토마토소스/케찹
석박지/깍두기
Tonic/ 토닉 워터
파마산치즈 기호 /
멸치육수/천연조미료육수
동그랑땡/함박스테이크
다진 마늘 /
소금/후추/이탈리안시즈닝
한우/소고기
다진파/다진 풋고추
깨/파슬리가루
통곡물가루/미숫가루
로메인/상추/양상추등
청/홍고추 각
물/육수
체다/고다 등 슬라이스 치즈
양배추/파프리카/고추
크레송/어린잎채소
코코아가루 / 시나몬가루
청주/고량주
Aardbeien/ 딸기
Bacon/Canadian Bacon
1/
삼치 반마리 뼈를 바른 삼치 / 길게
맛간장/쯔유
오미자액/매실액
다진마늘/통마늘
부추/파
실온/데운 우유
버터/올리브오일
할리피뇨/피클
맛간장/진간장/심미장
전복장간장/맛간장
청/ 홍고추 각
다시육수/물
방울토마토 노랑/빨강
피자/스파게티 소스
불고기/다짐육
청양/홍고추 각
파마산치즈가루/허브
햄/ 버섯/ 애호박/ 당근
홍고추/청고추
파프리카/양파
아몬드슬라이스/견과류
노랑/주황/빨강 파프리카
천연조미료육수/멸치육수
청/홍고추
조청/메이플시럽
청주/소주
그린/블랙 올리브
요거트/우유


#### 재료부분처리

'/'를 기준으로 두번째 데이터를 가져옵니다.

In [141]:
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '/' in data[0]:
                data[0] = data[0].split('/')[1].strip()

In [142]:
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '/' in data[0]:
                print(data)

In [143]:
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] != '':
                lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/56813 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\1993634024.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 56813/56813 [00:13<00:00, 4133.29it/s]


In [144]:
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if  data[0] == '':
                print(data)

#### 단위부분

In [145]:
for word in list(unit_slash_set):
    print(word)

1/2큰술 등 
3/1공기
1+1/2술
1/4종이컵
1+1/3숟갈
2/1공기
1/4ts
8/10컵
1+1/2줌
2/1T
1/3종이컵정도
1/4배
1/2씩
1/2조각
1/3병
11/2줌
2/3주걱
1/2모
1+1/2근
2/3개정도
1/3캔 
1/2절분량
1/2덩이
1/2칸술
받음/대파대체가능
1/2접시
1/2개 채썰어줌
1/3줌
1과1/2컵
1/2작은술
2/3머그컵
1/2찻술
1/2컵​
1/4통정도
1+1/2숟가락
2종1/4개씩
1+1/2숟갈
1/2대 분량
1/2그릇
1+1/2cc
1/6도막
1/6개 씩
2/3T
2/3통
1/2t스푼
3+1/2스푼
2스푼+1/2컵
1/2작은술​
1/4정도 
1/2줄
1/3토막
1/6조각
1/3게
1/2T
1/2숟갈
1/2폭
1/5모
1/3소주컵
1/2대정도
1/2대 또는 쪽파4대
1/2스폰
1/4작은술이내
1/4줄기
1/2에반병
2/3그릇 
1/2분량
1+1/2줄기
300g1/3팩
1/2잔
2공기 +1/2공기
약간/한꼬집
1+1/2뿌리
1/4개씩
1/4큰술
1/2대. 물1리터
1/2줌씩
2/3밥그릇
2/3ts
2/3차스푼
1TS +1/2ts
1/2컵 +a
1/2t 스푼
1+1/2작은숟가락
1./2개
1/4톨
1/8쪽
1/20개
4+1/2숟가락
1/23개
1/3정도
1/2L 정도
3/2컵
1/2묶음
2/3만큼
1/3새
1/3캔
1/2만
1/8포기
1.8리터2/3병
3/4cup
1/4개 당근
1/8토막
1/3개.
1/4tsp
1/6ts
1과1/4컵
200g1/4포기
1공기+1/2
3/2
1과1/2큰술.
1/2tsp
1/2소주잔
1/2근
1+1/2스쿱
1/4C씩
1/2개 물1/2컵
1/2스푼 
1개당1/2쪽
1+1/2공기
1/2손가락
1/2
1/4모100g
1/2국자
1+3/4oz
1과1/4컵 등
1/16개
10cm길이로1/2토막
1/4등분
1/2개 이하
2+1/2밥숟가락
1+1/3Ts
1+1/2모
1+1/2줌 
2/3큰 술
1/3가량 
2/3줌
1/2개125g
1/4쪽 씩
1/3t​
1장에1/4컵
1/2큰 술 대파1대
2/8쪽
3+

#### 단위부분처리

단위부분을 처리하는 함수는 다른 특수문자를 포함한 데이터를 처리한뒤에 수행하겠습니다.

In [146]:
import re
def func(word):
    word = word.split()[0]
    lst = []
    check = True
    for i in range(len(word)):
        try:
            int(word[i])
            break
        except:
            check = False
    if check:
        for i in range(len(word)):
            try:
                v = round(eval(word[:i+1]),1)
                lst.append(v)
            except:
                pass
        special = re.compile(r'[^ A-Za-z가-힣]')
        special2 = re.compile(r'[^ .0-9]')
        result = str(lst[-1]) + str(special.sub('', word))
        value = special2.sub('', result)
        unit = special.sub('', result)
        return [value, unit]
    else:
        return ['1', '단위']

In [147]:
 copy_data['ingredient'][12222]

[['채소', '1단위'], ['베이컨', '1단위'], ['계란', '1단위'], ['밥', '1단위']]

In [148]:
for data in copy_data['ingredient'][12222]:
    word = data[1]
    print(word)
    print(func(word))
    print('-'*10)

1단위
['1', '단위']
----------
1단위
['1', '단위']
----------
1단위
['1', '단위']
----------
1단위
['1', '단위']
----------


In [149]:
lst =[['당근','1개'], ['양파', '2개']]

for i in range(len(lst)):
    temp = []
    temp.append(lst[i][0])
    temp += func(lst[i][1])
    lst[i] = temp
lst

[['당근', '1', '개'], ['양파', '2', '개']]

### 우리가 설정한 특수문자외 다른 특수문자 제거

In [150]:
test ='!!ada'
result = re.sub(r'[^-~+/.0-9a-zA-Z가-힣 ]', '',test)
result

'ada'

In [151]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            data[0] = re.sub(r'[^-~+/.0-9a-zA-Z가-힣 ]', '', data[0]).strip()
            data[1] = re.sub(r'[^-~+/.0-9a-zA-Z가-힣 ]', '', data[1]).strip()

100%|██████████| 56813/56813 [00:01<00:00, 31479.40it/s]


In [152]:
special = re.compile(r'[-~+/.0-9a-zA-Z가-힣 ]')
result = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            result.add(special.sub('', data[0]))
            result.add(special.sub('', data[1]))

100%|██████████| 56813/56813 [00:01<00:00, 45650.01it/s]


### 마지막으로 공백문자 혹은 빈칸 확인

In [153]:
drop_index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data == []:
            drop_index.append(i)

100%|██████████| 56813/56813 [00:00<00:00, 106591.81it/s]


In [154]:
len(drop_index)

0

In [155]:
index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
            if data[0] =='' or data[1] =='':
                index.add(i)

100%|██████████| 56813/56813 [00:00<00:00, 92680.24it/s]


In [156]:
for i in tqdm(index):
    lst =[]
    for data in copy_data['ingredient'][i]:
        if data[0] == '' or data[1] == '':
            pass
        else:
            lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/42 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26724\2704427141.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 42/42 [00:00<00:00, 3818.21it/s]


In [157]:
drop_index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    if copy_data['ingredient'][i] == []:
        drop_index.add(i)

100%|██████████| 56813/56813 [00:00<00:00, 117869.97it/s]


In [158]:
copy_data = copy_data.drop(list(drop_index))
copy_data = copy_data.reset_index(drop=True)
copy_data.head()

,id,ingredient,only_ingred
0,128671,"[[어묵, 2개], [김밥용김, 3장], [당면, 1움큼], [양파, 1/2개], ...","[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,131871,"[[현미, 4컵], [찹쌀, 2컵], [호두, 50g], [물, 1/2컵], [소금...","[현미, 찹쌀, 호두, 물, 소금]"
2,139247,"[[북어포, 1마리], [찹쌀가루, 1C]]","[북어포, 찹쌀가루]"
3,149207,"[[파스타면, 1단위]]",[파스타면]
4,151148,"[[건표고버섯, 9개], [오이, 1/2개], [당근, 1/2개], [양파, 1/2...","[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


In [159]:
drop_index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    if copy_data['ingredient'][i] == []:
        drop_index.add(i)

100%|██████████| 56744/56744 [00:00<00:00, 117914.91it/s]


In [160]:
len(drop_index)

0

### 위에서 언급했던 단위부분처리('+','/')하기

In [161]:
lst =[['당근','1개'], ['양파', '2개']]

for i in range(len(lst)):
    temp = []
    temp.append(lst[i][0])
    temp += func(lst[i][1])
    lst[i] = temp
lst

[['당근', '1', '개'], ['양파', '2', '개']]

In [162]:
all_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        temp = []
        temp.append(data[0])
        temp += func(data[1])
        lst.append(temp)
    all_lst.append(lst)

100%|██████████| 56744/56744 [00:11<00:00, 4759.88it/s]


In [163]:
copy_data['ingredient'] = pd.Series(all_lst)
copy_data['ingredient']

0        [[어묵, 2, 개], [김밥용김, 3, 장], [당면, 1, 움큼], [양파, 0...
1        [[현미, 4, 컵], [찹쌀, 2, 컵], [호두, 50, g], [물, 0.5,...
2                             [[북어포, 1, 마리], [찹쌀가루, 1, C]]
3                                          [[파스타면, 1, 단위]]
4        [[건표고버섯, 9, 개], [오이, 0.5, 개], [당근, 0.5, 개], [양...
                               ...                        
56739                           [[감자, 1, 개], [대파, 0.2, 개]]
56740    [[양파, 0.3, 개], [배추, 2, 잎], [당근, 1, 단위], [청경채, ...
56741             [[순대, 400, g], [청양고추, 1, 개], [계란, 3, 개]]
56742    [[양배추, 0.2, 통], [양파, 0.5, 개], [편마늘, 1, 숟가락], [...
56743    [[식빵, 4, 장], [감자, 2, 개], [양파, 0.2, 개], [피망, 0....
Name: ingredient, Length: 56744, dtype: object

## 주요 재료 단위 통일

가격산정을 위하여 처리해야할 주요 재료를 선정하고, 주요 재료만 단위를 통일하겠습니다.

단위의 수를 세는 함수입니다.

In [ ]:
def count_unit(word):
    unit_list = []
    for i in range(len(copy_data['ingredient'])):
        for data in copy_data['ingredient'][i]:
            if data[0] == word:
                unit = data[2]
                unit_list.append(unit)
    counter = Counter(unit_list)

단위를 변경할 재료 리스트입니다.

In [168]:
ingredient_list = ['감귤','귤','김','팥','방울토마토','포도','쇠고기','소고기','팥','쌀','찹쌀','콩','녹두','고구마','감자','배추','양배추','상추',
                   '오이','호박','토마토','딸기','무','당근','건고추','풋고추','붉은고추','마늘','양파','파','생강','미나리',
                   '깻잎','피망','파프리카','멜론','땅콩','느타리버섯','팽이버섯','새송이버섯','사과','배','단감','바나나',
                   '돼지고기','닭고기','계란','달걀','명태','건멸치','건오징어','건미역','굴','조기','호두','아몬드',
                   '레몬','건포도','파인애플','새우','김치','즉석밥','두부','맛김','조미김','콩나물','얼갈이배추','수박','우유','고등어',
                   '건블루베리','참다래','전복','꽁치','시금치','열무','오렌지','갈치','물오징어','오징어','망고','가지','부추','참외',
                   '아보카도','체리','복숭아'
                    ]
for ingredient in ingredient_list:
    count_unit(ingredient)

감귤: [('개', 6), ('단위', 2)]
귤: [('개', 63), ('단위', 32), ('g', 2)]
김: [('장', 545), ('단위', 466), ('봉지', 14)]
팥: [('단위', 20), ('컵', 19), ('g', 6)]
방울토마토: [('개', 539), ('단위', 190), ('알', 50)]
포도: [('단위', 14), ('알', 10), ('송이', 9)]
쇠고기: [('g', 148), ('단위', 20), ('줌', 6)]
소고기: [('g', 679), ('단위', 193), ('줌', 28)]
팥: [('단위', 20), ('컵', 19), ('g', 6)]
쌀: [('컵', 260), ('단위', 78), ('g', 69)]
찹쌀: [('컵', 106), ('단위', 47), ('g', 19)]
콩: [('단위', 10), ('컵', 9), ('g', 2)]
녹두: [('단위', 11), ('컵', 11), ('g', 2)]
고구마: [('개', 491), ('단위', 132), ('g', 71)]
감자: [('개', 2503), ('단위', 442), ('g', 127)]
배추: [('장', 67), ('단위', 61), ('g', 29)]
양배추: [('단위', 690), ('g', 302), ('개', 250)]
상추: [('장', 409), ('단위', 246), ('g', 30)]
오이: [('개', 1903), ('단위', 672), ('g', 53)]
호박: [('개', 409), ('단위', 383), ('g', 47)]
토마토: [('개', 881), ('단위', 250), ('g', 23)]
딸기: [('개', 267), ('단위', 104), ('알', 27)]
무: [('단위', 367), ('개', 366), ('g', 300)]
당근: [('개', 2918), ('단위', 2668), ('g', 578)]
건고추: [('개', 75), ('단위', 17), ('g', 2)]
풋고추: [

주요 재료들은 아래와 같이 단위를 통일하겠습니다.

In [169]:
ingre_dict = {
            '감귤': '개',
            '귤': '개',
            '김': '장',
            '팥': '컵',
            '방울토마토': '개',
            '포도': '송이',
            '쇠고기': 'g',
            '소고기': 'g',
            '팥': '컵',
            '쌀': '컵',
            '찹쌀': '컵',
            '콩': '컵', 
            '녹두': '컵',
            '고구마':'개',
            '감자': '개',
            '배추': '포기',
            '양배추': '포기',
            '상추': '장',
            '오이': '개',
            '호박': '개',
            '토마토': '개',
            '딸기': '개',
            '무': '개',
            '당근': '개',
            '건고추': '개',
            '풋고추': '개',
            '붉은고추': '개',
            '마늘': '개',
            '양파': '개',
            '파': '개',
            '생강': '개',
            '미나리': 'g',
            '깻잎': '장',
            '피망': '개',
            '파프리카': '개',
            '멜론': '개',
            '땅콩': '개', 
            '느타리버섯': '개',
            '팽이버섯': '봉지',
            '새송이버섯': '개',
            '사과': '개',
            '배': '개',
            '단감': '개',
            '바나나': '개',
            '돼지고기': 'g',
            '닭고기': 'g',
            '계란': '개',
            '달걀': '개',
            '명태': '마리',
            '건멸치': 'g',
            '건오징어': '마리',
            '건미역': 'g', 
            '굴': 'g',
            '조기': '마리',
            '호두': 'g',
            '아몬드': 'g',
            '레몬': '개',
            '건포도': 'g',
            '파인애플': '개',
            '새우': '마리',
            '김치': '포기',
            '즉석밥': '개',
            '두부': '모',
            '맛김': '장',
            '조미김': '장',
            '콩나물': 'g',
            '얼갈이배추': '단',
            '수박': '개',
            '우유': 'ml',
            '고등어': '마리',
            '건블루베리': 'g',
            '참다래': '개',
            '전복': '마리',
            '꽁치': '마리',
            '시금치': '단',
            '열무': '단',
            '오렌지': '개',
            '갈치': '마리',
            '물오징어': '마리',
            '오징어': '마리',
            '망고': '개',
            '가지': '개',
            '부추': 'g',
            '참외': '개',
            '아보카도': '개',
            '체리': '개',
            '복숭아': '개'
              }

단위를 변경하는 함수입니다.

In [170]:
def change_unit(word):
    for i in range(len(copy_data['ingredient'])):
        for data in copy_data['ingredient'][i]:
            if data[0] == word:
                data[2] = ingre_dict[word]

In [171]:
for ingredient in tqdm(ingredient_list):
    change_unit(ingredient)

100%|██████████| 87/87 [00:44<00:00,  1.94it/s]


In [172]:
for ingredient in ingredient_list:
    count_unit(ingredient)

감귤: [('개', 8)]
귤: [('개', 99)]
김: [('장', 1059)]
팥: [('컵', 50)]
방울토마토: [('개', 849)]
포도: [('송이', 40)]
쇠고기: [('g', 184)]
소고기: [('g', 989)]
팥: [('컵', 50)]
쌀: [('컵', 529)]
찹쌀: [('컵', 212)]
콩: [('컵', 26)]
녹두: [('컵', 27)]
고구마: [('개', 703)]
감자: [('개', 3193)]
배추: [('포기', 248)]
양배추: [('포기', 2062)]
상추: [('장', 772)]
오이: [('개', 2722)]
호박: [('개', 923)]
토마토: [('개', 1201)]
딸기: [('개', 463)]
무: [('개', 1469)]
당근: [('개', 6753)]
건고추: [('개', 96)]
풋고추: [('개', 426)]
붉은고추: [('개', 64)]
마늘: [('개', 3946)]
양파: [('개', 16632)]
파: [('개', 3638)]
생강: [('개', 418)]
미나리: [('g', 413)]
깻잎: [('장', 1786)]
피망: [('개', 710)]
파프리카: [('개', 2008)]
멜론: [('개', 12)]
땅콩: [('개', 123)]
느타리버섯: [('개', 731)]
팽이버섯: [('봉지', 1009)]
새송이버섯: [('개', 1064)]
사과: [('개', 784)]
배: [('개', 270)]
단감: [('개', 45)]
바나나: [('개', 535)]
돼지고기: [('g', 970)]
닭고기: [('g', 144)]
계란: [('개', 7174)]
달걀: [('개', 5036)]
명태: [('마리', 2)]
건멸치: [('g', 13)]
건오징어: [('마리', 3)]
건미역: [('g', 52)]
굴: [('g', 376)]
조기: [('마리', 26)]
호두: [('g', 316)]
아몬드: [('g', 210)]
레몬: [('개', 305)]
건포도:

## 추천 시스템에 사용할 수 있게 데이터프레임 만들기
아래의 모양으로 데이터 프레임을 변경하도록 하겠습니다.

In [173]:
import pandas as pd

In [174]:
data = {"일련번호" :[123123, 345335, 53424],
        "재료" :  [[["양파", "2개"],['양파','2개'], ["당근", '3개'], ['밀가루', '100g']],[["브로콜리", '1개'], ['물', '1+1/2컵'], ['상추', '8~10장']], []]}

df = pd.DataFrame(data)
df

,일련번호,재료
0,123123,"[[양파, 2개], [양파, 2개], [당근, 3개], [밀가루, 100g]]"
1,345335,"[[브로콜리, 1개], [물, 1+1/2컵], [상추, 8~10장]]"
2,53424,[]


In [175]:
copy_data.head()

,id,ingredient,only_ingred
0,128671,"[[어묵, 2, 개], [김밥용김, 3, 장], [당면, 1, 움큼], [양파, 0...","[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,131871,"[[현미, 4, 컵], [찹쌀, 2, 컵], [호두, 50, g], [물, 0.5,...","[현미, 찹쌀, 호두, 물, 소금]"
2,139247,"[[북어포, 1, 마리], [찹쌀가루, 1, C]]","[북어포, 찹쌀가루]"
3,149207,"[[파스타면, 1, 단위]]",[파스타면]
4,151148,"[[건표고버섯, 9, 개], [오이, 0.5, 개], [당근, 0.5, 개], [양...","[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


In [176]:
ingredient_set = set()
for li in tqdm(copy_data['ingredient']):
    for data in li:
        ingredient_set.add(data[0])

100%|██████████| 56744/56744 [00:00<00:00, 319531.70it/s]


In [177]:
len(ingredient_set)

21113

In [178]:
# ingredient_set = set()
# for li in df['재료']:
#   for data in li:
#     ingredient_set.add(data[0])

# ingredient_set

In [179]:
# numbers_dic = dict()
# for i in range(len(df)):
#   index = df.iloc[i, 0]
#   lst =[]
#   for li in df.loc[(df['일련번호']==index)]['재료']:
#     for data in li:
#       lst.append({data[0]:data[1]})
#   numbers_dic[index] = lst
# numbers_dic

In [180]:
numbers_dic = dict()
for i in tqdm(range(len(copy_data))):
    index = copy_data.iloc[i, 0]
    lst =[]
    for li in copy_data.loc[(copy_data['id']==index)]['ingredient']:
        for data in li:
            lst.append({data[0]:data[1]})
    numbers_dic[index] = lst
numbers_dic

100%|██████████| 56744/56744 [00:21<00:00, 2580.04it/s]


{128671: [{'어묵': '2'},
  {'김밥용김': '3'},
  {'당면': '1'},
  {'양파': '0.5'},
  {'당근': '0.5'},
  {'깻잎': '6'},
  {'튀김가루': '1'},
  {'올리브유': '1'},
  {'간장': '1'},
  {'참기름': '1'}],
 131871: [{'현미': '4'}, {'찹쌀': '2'}, {'호두': '50'}, {'물': '0.5'}, {'소금': '1'}],
 139247: [{'북어포': '1'}, {'찹쌀가루': '1'}],
 149207: [{'파스타면': '1'}],
 151148: [{'건표고버섯': '9'},
  {'오이': '0.5'},
  {'당근': '0.5'},
  {'양파': '0.5'},
  {'사과': '0.5'},
  {'그외의': '1'},
  {'과일': '1'}],
 153040: [{'닭': '1'}],
 221094: [{'양파': '1'}, {'빵가루': '1'}, {'밀가루': '1'}, {'물': '1'}, {'파슬리': '1'}],
 221097: [{'밥': '2'},
  {'밥': '2'},
  {'참치캔 큰거': '1'},
  {'양파': '1'},
  {'다진김치': '1'},
  {'김밥용김': '1'}],
 221098: [{'떡볶이떡': '150'},
  {'당근': '0.5'},
  {'파': '1'},
  {'파': '1'},
  {'양파': '0.5'},
  {'양배추': '0.2'},
  {'어묵': '2'},
  {'오징어': '1'},
  {'새우': '1'},
  {'조갯살': '1'},
  {'홍합살': '1'}],
 221101: [{'스팸 싱글': '1'}, {'토마토': '1'}, {'상추': '2'}, {'치즈': '1'}],
 221102: [{'밥': '2'}, {'깻잎': '10'}, {'참치 작은캔': '1'}, {'당근': '0.5'}],
 223583: [{'새우': '10'},
  {'밥': 

In [182]:
# new = pd.concat([df['일련번호'], pd.DataFrame(columns=ingredient_set)], axis=1)
# new

In [183]:
new = pd.concat([copy_data['id'], pd.DataFrame(columns=ingredient_set)], axis=1)
new.head()

,id,라이스페이퍼,토마토소스 크게,샘표,콩나물 또는 숙주,다짐 소고기 볶음,흑미식빵,불린 콩물,애호박 중지,화이트초콜렛,...,튀김가루 듬뿍,청 치커리,인삼분말,간 소금,냉동볶음우동,커스타드 믹스,코팅용 초콜릿,서브빵,CU,생크림소스
0,128671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,131871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,139247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,149207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,151148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
for i in tqdm(range(len(copy_data))):
    index = copy_data.iloc[i, 0]
    for ingredient in list(ingredient_set):
        for li in numbers_dic[index]:
            try:
                value=li[ingredient]
                new.loc[(new['id']==index),ingredient]=float(value)
            except:
                pass

new = new.fillna(0)
new.head()

100%|██████████| 56744/56744 [53:24<00:00, 17.71it/s] 


,id,라이스페이퍼,토마토소스 크게,샘표,콩나물 또는 숙주,다짐 소고기 볶음,흑미식빵,불린 콩물,애호박 중지,화이트초콜렛,...,튀김가루 듬뿍,청 치커리,인삼분말,간 소금,냉동볶음우동,커스타드 믹스,코팅용 초콜릿,서브빵,CU,생크림소스
0,128671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,131871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,139247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,149207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,151148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 다른 곳에서 사용할 수 있도록 저장

In [186]:
copy_data.to_parquet('ingredient_text_preprocessing2.parquet',index=False)

In [187]:
new.to_pickle('modeling_text_data2.pkl')

같은 노트북에서 코사인 유사도를 구하려면 위에서 언급한 MemoryError가 발생하여  
코사인 유사도를 구하는 과정은 다른 노트북에서 처리하겠습니다.